In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install earthengine-api
!pip install joblib

In [3]:
!pip install earthengine-api
!pip install google-auth

In [4]:
import ee
from datetime import date
from joblib import dump

# Path to your service account JSON file in Google Drive
service_account_json = '/content/drive/My Drive/Learning/ee-penguincynthia-d20a99054f23.json'

# Authenticate and initialize using the service account JSON
credentials = ee.ServiceAccountCredentials(None, service_account_json)
ee.Initialize(credentials)

print("Google Earth Engine authenticated and initialized successfully!")

*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_0JLhFqfSY1uiEaW?source=Init


Google Earth Engine authenticated and initialized successfully!


In [5]:
# Get today's date for dynamic filtering
today = date.today()
start_date = "2017-03-01"
end_date = today.strftime("%Y-%m-%d")  # Dynamic end date

print(f"Using date range: {start_date} to {end_date}")

Using date range: 2017-03-01 to 2024-11-30


In [ ]:
# Load administrative boundaries for Trans-Nzoia
admin_boundaries = ee.FeatureCollection("FAO/GAUL/2015/level2")
trans_nzoia = admin_boundaries.filter(ee.Filter.eq("ADM2_NAME", "Trans Nzoia"))

# Load Sentinel-2 image collection and clip to Trans-Nzoia
sentinel2 = ee.ImageCollection("COPERNICUS/S2") \
    .filterDate(start_date, end_date) \
    .filterBounds(trans_nzoia) \
    .map(lambda image: image.clip(trans_nzoia))

# Load Sentinel-1 image collection and clip to Trans-Nzoia
sentinel1 = ee.ImageCollection("COPERNICUS/S1_GRD") \
    .filterDate(start_date, end_date) \
    .filterBounds(trans_nzoia) \
    .filter(ee.Filter.listContains("transmitterReceiverPolarisation", "VV")) \
    .filter(ee.Filter.listContains("transmitterReceiverPolarisation", "VH")) \
    .filter(ee.Filter.eq("instrumentMode", "IW")) \
    .map(lambda image: image.clip(trans_nzoia))

print("Data loaded successfully!")

Data loaded successfully!


/usr/local/lib/python3.10/dist-packages/ee/deprecation.py:207: DeprecationWarning: 

Attention required for COPERNICUS/S2! You are using a deprecated asset.
To ensure continued functionality, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2

  warnings.warn(warning, category=DeprecationWarning)


In [ ]:
# Define NDVI and EVI indices for Sentinel-2
def add_indices(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    evi = image.expression(
        "2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 10000))", {
            'NIR': image.select('B8'),
            'RED': image.select('B4'),
            'BLUE': image.select('B2')
        }).rename('EVI')
    return image.addBands([ndvi, evi])

# Define RVI index for Sentinel-1
def add_rvi(image):
    rvi = image.expression(
        "sqrt(vv / (vv + vh)) * (vv / vh)", {
            'vv': image.select('VV'),
            'vh': image.select('VH')
        }).rename('RVI')
    return image.addBands(rvi)

print("Index functions defined successfully!")

Index functions defined successfully!


In [ ]:
# Apply indices to Sentinel-2 and Sentinel-1
sentinel2_with_indices = sentinel2.map(add_indices)
sentinel1_with_rvi = sentinel1.map(add_rvi)

# Combine the two collections
combined_collection = sentinel2_with_indices.merge(sentinel1_with_rvi)
print("Indices added and collections combined successfully!")

Indices added and collections combined successfully!


In [ ]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np
from datetime import date

# Get today's date for dynamic filtering
today = date.today()
start_date = "2017-03-01"
end_date = today.strftime("%Y-%m-%d")  # Dynamic end date

# Simulated Data for Plotting
# Replace this with real data extracted from your GEE collection
dates = pd.date_range(start=start_date, end=end_date, freq="M")
ndvi_values = np.random.uniform(0, 1, len(dates))  # Simulated NDVI
evi_values = np.random.uniform(0, 1, len(dates))   # Simulated EVI
rvi_values = np.random.uniform(0, 1, len(dates))   # Simulated RVI

# Create a DataFrame for better handling
data = pd.DataFrame({
    'Date': dates,
    'Year': dates.year,
    'Month': dates.month_name(),
    'NDVI': ndvi_values,
    'EVI': evi_values,
    'RVI': rvi_values
})

# Filter Data by Dynamic Start and End Date
filtered_data = data[(data['Date'] >= pd.Timestamp(start_date)) &
                     (data['Date'] <= pd.Timestamp(end_date))]

# Plotly Interactive Visualization
fig = go.Figure()

# Add NDVI Line
fig.add_trace(go.Scatter(
    x=filtered_data['Date'],
    y=filtered_data['NDVI'],
    mode='lines+markers',
    name='NDVI',
    line=dict(color='green'),
    text=filtered_data['Month'] + " " + filtered_data['Year'].astype(str),  # Hover text
    hovertemplate='Month: %{text}<br>NDVI: %{y:.2f}<extra></extra>'
))

# Add EVI Line
fig.add_trace(go.Scatter(
    x=filtered_data['Date'],
    y=filtered_data['EVI'],
    mode='lines+markers',
    name='EVI',
    line=dict(color='blue'),
    text=filtered_data['Month'] + " " + filtered_data['Year'].astype(str),  # Hover text
    hovertemplate='Month: %{text}<br>EVI: %{y:.2f}<extra></extra>'
))

# Add RVI Line
fig.add_trace(go.Scatter(
    x=filtered_data['Date'],
    y=filtered_data['RVI'],
    mode='lines+markers',
    name='RVI',
    line=dict(color='red'),
    text=filtered_data['Month'] + " " + filtered_data['Year'].astype(str),  # Hover text
    hovertemplate='Month: %{text}<br>RVI: %{y:.2f}<extra></extra>'
))

# Update Layout for Better Visualization
fig.update_layout(
    title='Interactive Plot of NDVI, EVI, and RVI',
    xaxis_title='Date',
    yaxis_title='Index Value',
    xaxis=dict(
        tickformat='%b %Y',  # Format ticks as "Month Year"
        showgrid=True
    ),
    yaxis=dict(
        showgrid=True
    ),
    hovermode='x unified',  # Shows hover info for all traces at once
    legend=dict(
        title='Indices',
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="center",
        x=0.5
    )
)

# Show the Plot
fig.show()

<ipython-input-13-fabc5bef572a>:13: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  dates = pd.date_range(start=start_date, end=end_date, freq="M")


In [ ]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np
from datetime import date

# Get today's date for dynamic filtering
today = date.today()

# Default Start and End Dates
default_start_date = "2017-03-01"
default_end_date = today.strftime("%Y-%m-%d")  # Current date as default end

# Simulated Data for Plotting
# Replace this with real data extracted from your GEE collection
dates = pd.date_range(start=default_start_date, end=default_end_date, freq="M")
ndvi_values = np.random.uniform(0, 1, len(dates))  # Simulated NDVI
evi_values = np.random.uniform(0, 1, len(dates))   # Simulated EVI
rvi_values = np.random.uniform(0, 1, len(dates))   # Simulated RVI

# Create a DataFrame for better handling
data = pd.DataFrame({
    'Date': dates,
    'Year': dates.year,
    'Month': dates.month_name(),
    'NDVI': ndvi_values,
    'EVI': evi_values,
    'RVI': rvi_values
})

# Dynamic User Input for Start and End Dates
print("Enter the start and end dates for filtering the data:")
start_date = input(f"Start Date (YYYY-MM-DD) [Default: {default_start_date}]: ") or default_start_date
end_date = input(f"End Date (YYYY-MM-DD) [Default: {default_end_date}]: ") or default_end_date

# Validate Date Inputs
try:
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)
    if start_date > end_date:
        raise ValueError("Start date cannot be later than end date.")
except Exception as e:
    print(f"Invalid date input: {e}. Using default dates.")
    start_date = pd.to_datetime(default_start_date)
    end_date = pd.to_datetime(default_end_date)

# Filter Data Based on Input Dates
filtered_data = data[(data['Date'] >= start_date) &
                     (data['Date'] <= end_date)]

# Plotly Interactive Visualization
fig = go.Figure()

# Add NDVI Line
fig.add_trace(go.Scatter(
    x=filtered_data['Date'],
    y=filtered_data['NDVI'],
    mode='lines+markers',
    name='NDVI',
    line=dict(color='green'),
    text=filtered_data['Month'] + " " + filtered_data['Year'].astype(str),  # Hover text
    hovertemplate='Month: %{text}<br>NDVI: %{y:.2f}<extra></extra>'
))

# Add EVI Line
fig.add_trace(go.Scatter(
    x=filtered_data['Date'],
    y=filtered_data['EVI'],
    mode='lines+markers',
    name='EVI',
    line=dict(color='blue'),
    text=filtered_data['Month'] + " " + filtered_data['Year'].astype(str),  # Hover text
    hovertemplate='Month: %{text}<br>EVI: %{y:.2f}<extra></extra>'
))

# Add RVI Line
fig.add_trace(go.Scatter(
    x=filtered_data['Date'],
    y=filtered_data['RVI'],
    mode='lines+markers',
    name='RVI',
    line=dict(color='red'),
    text=filtered_data['Month'] + " " + filtered_data['Year'].astype(str),  # Hover text
    hovertemplate='Month: %{text}<br>RVI: %{y:.2f}<extra></extra>'
))

# Update Layout for Better Visualization
fig.update_layout(
    title=f'Interactive Plot of NDVI, EVI, and RVI ({start_date.date()} to {end_date.date()})',
    xaxis_title='Date',
    yaxis_title='Index Value',
    xaxis=dict(
        tickformat='%b %Y',  # Format ticks as "Month Year"
        showgrid=True
    ),
    yaxis=dict(
        showgrid=True
    ),
    hovermode='x unified',  # Shows hover info for all traces at once
    legend=dict(
        title='Indices',
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="center",
        x=0.5
    )
)

# Show the Plot
fig.show()

<ipython-input-14-272a0ac9ec6d>:15: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.



Enter the start and end dates for filtering the data:


In [ ]:
 # Save the processed collection as a model using joblib
model_path = '/content/drive/My Drive/Learning/gee_analysis_model.joblib'
dump(combined_collection, model_path)

print(f"Model saved successfully at {model_path}!")

Model saved successfully at /content/drive/My Drive/Learning/gee_analysis_model.joblib!
